# Data Aggregation
As a first step in the Places to Go Demo, we will need static venue data to create reccomendations from. In production, our venue sources will be managed by a Web Scraper bot that will handle crawling social media and updating the list based on activity. For now, we will populate a static set of 2,500 locations, which will be sourced from 5 cities. 

The five cities that have been requested by the client for the demo are:
- **New York**
- **Scottsdale**
- **Miami**
- **Los Angeles**
- **Chicago**

We will use the Yelp API to gather the top 500 rated locations in each city. We will then feed the `name` and `categories` field of each response to the AI model, which will seek to associate each venue with a list of keywords.

We will need to take the following steps to achieve our task:
1. Gather JSON objects for top 500 locations in each city
2. Extract exhaustive list of all categories from the 2,500 locations
3. Provide list of ChatGPT and prompt it to create a list of 20 keywords for each archetype
4. Design prompt for associating businesses with keywords based on `name` and `categories` field 
5. Run list of 2,500 businesses and store results in a JSON file.

In [6]:
import os
from dotenv import load_dotenv
load_dotenv("../.env")

YELP_API_KEY = os.getenv("YELP_API_KEY")

## 1. Gather JSON Data of Locations
We want to start by using the `/businesses/search` endpoint of the Yelp Fusion API to gather the top 500 rated locations in each of our 5 cities. We will store these responses directly in JSON files to retrieve for future steps.

In [23]:
import requests
from tqdm import tqdm

# List of cities to search
CITIES = ["New%20York%20City", "Scottsdale", "Miami", "Los%20Angeles", "Chicago"]
CITY_CODES = ["NYC", "SCOTTSDALE", "MIAMI", "LA", "CHICAGO"]

CITY_TO_CODE = dict(zip(CITIES, CITY_CODES))

# Yelp Fusion API URL
API_URL = "https://api.yelp.com/v3"
BUSINESS_SEARCH_ENDPOINT = "/businesses/search"

# Search Params For API Request
LIMIT = 50
SORT_BY = "rating"

# Authorization
HEADERS = {
    "Authorization": "Bearer " + YELP_API_KEY,
}

def request_city_data(city: str):
    """Request data from Yelp API for a given city"""
    base_url = f"{API_URL}{BUSINESS_SEARCH_ENDPOINT}?location={city}&limit={LIMIT}&sort_by={SORT_BY}"
    url = base_url + "&offset={}"
    offset = 0
    data = []
    for i in range(10):
        results = requests.get(url.format(offset), headers=HEADERS).json()
        
        # Add the city code to the data
        for result in results['businesses']: result['city'] = city

        offset += LIMIT
        data.extend(results["businesses"])
    return data


def extract_location_data():
    """Extract location data from Yelp API"""
    data = []
    for city in tqdm(CITIES):
        try:
            print("Requesting data for", city)
            city_results = request_city_data(city)
            data.extend(city_results)
            print(f"Received {len(city_results)} results for {city}. Total: {len(data)}")
        except Exception as e:
            print("Failed to request data for", city)
            print(e)
    return data

In [21]:
data = extract_location_data()

  0%|          | 0/5 [00:00<?, ?it/s]

Requesting data for New%20York%20City


 20%|██        | 1/5 [00:06<00:25,  6.48s/it]

Received 500 results for New%20York%20City. Total: 500
Requesting data for Scottsdale


 40%|████      | 2/5 [00:13<00:20,  6.71s/it]

...

Received 500 results for Miami. Total: 1500
Requesting data for Los%20Angeles


 80%|████████  | 4/5 [00:30<00:08,  8.08s/it]

Received 500 results for Los%20Angeles. Total: 2000
Requesting data for Chicago


100%|██████████| 5/5 [00:37<00:00,  7.57s/it]

Received 500 results for Chicago. Total: 2500


In [24]:
import json

with open("../data/raw_location_data.json", "w", encoding="utf-8") as f:
    for locaction in data:
        locaction['city_code'] = CITY_TO_CODE[locaction['city']]
    json.dump(data, f, ensure_ascii=False, indent=4)

## 2. Extract Exhuastive Category List from Locations
We now want to get an exhaustive list of all the categories provided in our 2,500 locaitons. To do this, we will extract the `categories` field from each location, and append the values to a list. Once all values have been appended, we will type cast the list to a set to remove duplicates. 

We will store the category list in a JSON file for future use.

In [25]:
categories = []
for location in data:
    loc_categories = [category['alias'] for category in location['categories']]
    categories.extend(loc_categories)
categories = list(set(categories))


with open("../data/raw_categories.json", "w", encoding="utf-8") as f:
    json.dump(categories, f, ensure_ascii=False, indent=4)

## 3. Keyword List Generation
We have used ChatGPT to convert our categories into keywords to use for classification. The keywords can be found in `../data/keywords.json`.

## 4. Prompt Engineering
Now that we have our keywords set, we need to do some prompt engineering to create a GPT-3.5-Turbo prompt which associates a venue with a set of our keywords. To do this, there are a few considerations we must make:
- Prompt must provide the list of keywords to the model
- Model must accurately associate keywords with venues according to product needs
- Want to process as many venues in one prompt as possible

To provide the LLM with the list of keywords, we will simply provide them in the system prompt. For token efficiency, we may try to cram as many venues as possible into every prompt, so we limit the number of times we have to send a system prompt.

In order to get accurate results without fine-tuning, we should take a few-shot approach, to do this, we will use ChatGPT to do ~20 locations, and we will then use these as an example for each prompt we send.

Finally, we should try to jam as many tokens as possible into each prompt. We have 16k tokens to work with as a context window. We can use the examples to determine the optimal number of locations to use per prompt.

In [73]:
with open("../data/keywords.json", "r", encoding="utf-8") as f:
    keywords = json.load(f)
    raw_keywords = []
    for cat, keyword in keywords.items():
        words = [word.replace("-", " ") for word in keyword]
        words = [word.title().replace(" ", "") for word in words]
        raw_keywords.extend(words)
    keyword_set = set(raw_keywords)
    

with open("../data/example.json", 'r', encoding='utf-8') as f:
    example = json.load(f)

def check_outputs(outputs):
    output_keywords = []
    for output in outputs['venues']:
        output_keywords.extend(output['keywords'])

    keyword_set = set(output_keywords)
    # Ensure that the keywords are all contained in the raw keywords
    if not keyword_set.issubset(raw_keywords):
        raise ValueError("output_keywords is not a subset of raw_keywords")

check_outputs(example['output'])


In [117]:
with open("../data/keywords.json", "r", encoding="utf-8") as f:
    keywords = json.load(f)
    keyword_list = []
    for cat, keyword in keywords.items():
        words = [word.replace("-", " ") for word in keyword]
        words = [word.title().replace(" ", "") for word in words]
        keywords[cat] = words
        keyword_list.extend(words)

In [134]:
with open("../data/location_data.json", "r", encoding="utf-8") as f:
    location_data = json.load(f)
    trimmed_location_data = []
    for location in location_data:
        trimmed_location_data.append({
            "id": location['id'],
            "name": location['name'],
            "city": location['city_code'],
            'rating': location['rating'],
            "categories": [category['title'] for category in location['categories']]
        })

# Add an embed term to each location
for location in trimmed_location_data:
    categories = location['categories']
    term = f"{location['name']}: {', '.join(categories)}"
    location['embed_term'] = term
    del location['categories']

In [124]:
from typing import List

vector_data = []

def embed_terms(terms: List[str]):
    response = client.embeddings.create(
        input=terms,
        model='text-embedding-ada-002'
    )
    return [datum.embedding for datum in response.data]


In [ ]:
keyword_embeddings = embed_terms(raw_keywords)
for keyword, embedding in zip(raw_keywords, keyword_embeddings):
    vector_data.append({
        "keyword": keyword,
        "vector": embedding
    })

In [126]:
import numpy as np
import pandas as pd

for vector in vector_data:
    vector['vector'] = np.array(vector['vector'])

df = pd.DataFrame(vector_data)

(60, 2)

In [127]:

# Now, we want to create an embedding for each venue, by embedding the keywords
terms = [location['embed_term'] for location in trimmed_location_data]
embeddings = [*embed_terms(terms[:1000]), *embed_terms(terms[1000:2000]), *embed_terms(terms[2000:])]
len(embeddings)

2500

In [137]:
cypher_entities = []
for i, data in enumerate(zip(embeddings, trimmed_location_data)):
    embed, location = data
    embed = np.array(embed)
    vector_store = df.copy()
    vector_store['similarity'] = df['vector'].apply(lambda x: np.dot(x, embed))
    vector_store = vector_store.sort_values(by='similarity', ascending=False)
    results = vector_store.keyword.values[:3].tolist()

    cypher_entities.append({
        'venue': {
            'id': location['id'],
            'name': location['name'],
            'city': location['city'],
            'rating': location['rating'],
        },
        'keywords': results,
    })
cypher_entities[0]

{'venue': {'id': 'LslssTS75mcFf-6pttxKBQ',
  'name': 'Panetino Bakery',
  'city': 'NYC',
  'rating': 5.0},
 'keywords': ['Coffee Culture', 'Baking Passion', 'Gourmet']}

In [138]:
with open("../data/cypher_entites.json", "w", encoding="utf-8") as f:
    json.dump(cypher_entities, f, ensure_ascii=False, indent=4)